# Analysis Example

Before running any analysis code in the notebook make sure that
test files were downloaded. To download the files open notebook
get_data.ipynb in the config directory. and run the commands in
the notebook.

In [ ]:
//---- imports for HIPO4 library
import org.jlab.jnp.hipo4.io.*;
import org.jlab.jnp.hipo4.data.*;
//---- imports for GROOT library
import org.jlab.groot.data.*;
import org.jlab.groot.graphics.*;
//---- imports for PHYSICS library
import org.jlab.jnp.physics.*;


HipoReader reader = new HipoReader(); // Create a reader obejct
//../config/run_004013_00_2c.hipo
reader.open("/Users/biselli/Data/clas12/rgB/pass1/dst_edeut_06321_trimmed.hipo"); // open a file

Now we create an Event class and a bank instance, in this case 
for particle bank. The event instance is used to read events from
the file, and Bank instance is used to read particle bank from each event.
NOTE ! the file must be opened first in order to initialize Bank object
since it takes the schema for the bank from file dictionary.

In [ ]:
Event     event = new Event();
Bank  particles = new Bank(reader.getSchemaFactory().getSchema("REC::Particle"));

Now we can loop over the events in the file and read particle bank and check
for electron in the first raw, if there is one we can create lorentz vector for the
particle and calculate W2 and Q2, and plot it. First we will declare histogram objects
and canvas object:

In [ ]:
H1F  hW = new H1F("hW" ,100, 0.5, 4.0);
H1F hQ2 = new H1F("hQ2",100, 0.1, 4.0);
hW.setTitleX("W [GeV]");
hQ2.setTitleX("Q^2 [GeV/c^2]");

EmbeddedCanvas ec = new EmbeddedCanvas(800,400);

Now we can loop over the events and count how many 
events we have where electron is detected.

In [ ]:
int counter = 0;
int    elec = 0;

while(reader.hasNext()==true){
    reader.nextEvent(event);
    event.read(particles);
    if(particles.getRows()>0){
        int pid = particles.getInt("pid",0);
        if(pid==11){
           elec++;
        }
    }
    counter++;
}
System.out.println("processed # " + counter + " , electrons : " + elec);

In [ ]:
LorentzVector  vBeam   = new LorentzVector(0.0,0.0,10.6,10.6);
LorentzVector  vTarget = new LorentzVector(0.0,0.0,0.0,0.938);
LorentzVector electron = new LorentzVector();
LorentzVector       vW = new LorentzVector();
LorentzVector      vQ2 = new LorentzVector();

reader.getEvent(event,0); // Reads the first event and resets to the begining of the file

while(reader.hasNext()==true){
    reader.nextEvent(event);
    event.read(particles);
    if(particles.getRows()>0){
        int pid = particles.getInt("pid",0);
        if(pid==11){
           electron.setPxPyPzM(
               particles.getFloat("px",0), 
               particles.getFloat("py",0),
               particles.getFloat("pz",0),
               0.0005
               );
           
           vW.copy(vBeam);
           vW.add(vTarget).sub(electron);
           
           vQ2.copy(vBeam);
           vQ2.sub(electron);
           
           hW.fill(vW.mass());
           hQ2.fill(-vQ2.mass2());
        }
    }
}

ec.divide(2,1);

ec.cd(0).draw(hW);
ec.cd(1).draw(hQ2);

ec.getScreenShot();